# Udiddit Social Network

Udiddit, a social news aggregation, web content rating, and discussion website, is currently using a risky and unreliable Postgres database schema to store the forum posts, discussions, and votes made by their users about different topics.

The schema allows posts to be created by registered users on certain topics, and can include a URL or a text content. It also allows registered users to cast an upvote (like) or downvote (dislike) for any forum post that has been created. In addition to this, the schema also allows registered users to add comments on posts.

## Initial Schema

### bad_posts (Just 5 post)

| id | topic        | username      | title             | url                  | text_content                      | upvotes                                | downvotes                                     |
|----|--------------|---------------|-------------------|----------------------|---------------------------------- |----------------------------------------|-----------------------------------------------|
| 1  | Synergized   | Gus32         | numquam iuand...  |                      | Voluptate ut similique libero ... | Judah.Okuneva94,Dasia98,...  | Lambert.Buckridge0,Joseph_Pouros82,Jesse_Yost |
| 2  | Applications | Keagan_Howell | officia tempo...  | http://lesley.com    |                                   | Marcellus31,Amina_Larson,... | Aniyah_Balistreri68,Demarcus.Berge,...        |
| 3  | Plastic      | Tia.Bosco     | officiis accu...  | http://aurelie.name  |                                   | Evangeline.Koss65,...        | Clement57,Hoyt79                              |
| 4  | Cloned       | Larue_Bayer22 | quos voluptat...  | http://gerry.info    |                                   | Lowell.Gutkowski3, ...       | Myra_Kunde91,Devin_Luettgen42,Deshawn.Cormier |
| 5  | Denmark      | Chesley.Will22| officia nihil...  |                      | Deleniti et tempore aut. Est ...  | Rylan_Strosin90, ...         | Brandi.Schaefer,German60,Vicky_Hill,...       | 


#### DDL used to create the above schema
    CREATE TABLE bad_posts (
        id SERIAL PRIMARY KEY,
        topic VARCHAR(50),
        username VARCHAR(50),
        title VARCHAR(150),
        url VARCHAR(4000) DEFAULT NULL,
        text_content TEXT DEFAULT NULL,
        upvotes TEXT,
        downvotes TEXT
    );


### bad_comments (Just 5 comments)

| id    | username                | post_id       | text_content                                                             |
|-------|-------------------------|-------------- |---------------------------------- ---------------------------------------|
| 1     | Liliane.Lakin40         | 3345          | Atque quaerat et. Omnis consequatur qui possimus sit accusantium dic...  |
| 2     | Adeline99               | 2615          | Ab ea ad velit tempore. Consectetur quis corporis modi id. Rerum dol...  |
| 3     | Darrel_Reynolds         | 9102          | Ratione facilis et beatae aut temporibus et qui nemo. Dolores offic...   |
| 4     | Kolby.Langosh           | 9734          | Veritatis doloribus officiis cupiditate quibusdam voluptatem facili...   |
| 5     | Jared_Koss45            | 6045          | Impedit alias magnam mollitia ratione nemo. Consequatur ratione quis...  |


#### DDL used to create the above schema
    CREATE TABLE bad_comments (
        id SERIAL PRIMARY KEY,
        username VARCHAR(50),
        post_id BIGINT,
        text_content TEXT
    );



## Investigate the existing schema

Here as some notes after investigating the above schema

1. Having the ***username*** in both tables is a bad idea. If the user decides to change his/her username, this would warrant going through all post and comments made by that user in order to effect the change. 
    A better approach would be having a **user_table** with an __id__ and then use the **user_id** instead.
    
    
2. ***upvotes*** and ***downvotes*** has like an array of users concatinated with a commma (,). This a bad approach for 2 reasons;
    - The problem raised above would consistitue a huge mess while trying to search for user in that array so you can effect a change made to the **username**.
    - It breaks the **First Normal Form** *which has a goal of having a single value in a cell*.
   
   
3. No **FOREIGN KEY** links between the **post_id**, in the **bad_comments** table, and the **id**, in the **bad_post** table. This means that a comment can be added for a post that doesn't exist.


4. **post_id** is **BIGINT** instead of **INTEGER** which is the data type of the **id** column in the **bad_post** table.


5. **topic** in the same table as post... Same issue with username.



## Creating a better schema for Udiddity Social Network

### Creating a users table 
    CREATE TABLE "users" (
        "id" SERIAL PRIMARY KEY,
        "username" VARCHAR(25) UNIQUE NOT NULL,
        CONSTRAINT "username_cant_be_empty" CHECK(LENGTH(TRIM("username")) > 0),
        "is_loggedIn" SMALLINT DEFAULT 1,
        CONSTRAINT "boolean_to_check_user_loggin_status" CHECK("is_loggedIn" IN (0, 1)),
        "last_login" TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    CREATE INDEX "fast_search_of_inactive_users" ON "users" ("is_loggedIn", DATE_PART('YEAR', "last_login"));

### Creating a topics table
    CREATE TABLE "topics" (
        "id" SERIAL PRIMARY KEY, 
        "name" VARCHAR(30) UNIQUE NOT NULL, 
        CONSTRAINT "name_cant_be_empty" CHECK(LENGTH(TRIM("name")) > 0),
        "description" VARCHAR(500)
    );
    
### Creating a posts table
    CREATE TABLE "posts" (
        "id" SERIAL PRIMARY KEY,
        "user_id" INTEGER REFERENCES "users" ON DELETE SET NULL,
        "topic_id" INTEGER REFERENCES "topics" ("id") ON DELETE CASCADE,
        "title" VARCHAR(100) NOT NULL,
        "url" VARCHAR(500),
        "text_content" TEXT,
        CONSTRAINT "cant_have_url_and_text_together" CHECK(("url" IS NULL AND "text_content" IS NOT NULL) OR ("url" IS NOT NULL AND "text_content" IS NULL)),
        "posted_on" TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    CREATE INDEX "fast_search_of_post_by_topic" ON "posts" ("topic_id");
    CREATE INDEX "fast_search_of_post_by_user" ON "posts" ("user_id");
    CREATE INDEX "fast_search_of_post_linked_to_url" ON "posts" ("url");
    
### Creating a comments table
     CREATE TABLE "comments" (
         "id" SERIAL PRIMARY KEY,
         "user_id" INTEGER REFERENCES "users" ON DELETE SET NULL,
         "post_id" INTEGER REFERENCES "posts" ("id") ON DELETE CASCADE,
         "text_content" TEXT NOT NULL,
         CONSTRAINT "text_content_cant_be_empty" CHECK(LENGTH(TRIM("text_content")) > 0),
         "comment_id" INTEGER REFERENCES "comments" ("id") ON DELETE CASCADE,
         "posted_on" TIMESTAMP DEFAULT CURRENT_TIMESTAMP
     );
     CREATE INDEX "fast_search_of_top_level_comments" ON "comments" ("comment_id");
     CREATE INDEX "fast_search_of_comments_by_user" ON "comments" ("user_id");
    
### Creating a votes table
    CREATE TABLE "votes" (
        "id" SERIAL PRIMARY KEY,
        "user_id" INTEGER REFERENCES "users" ON DELETE SET NULL,
        "post_id" INTEGER REFERENCES "posts" ("id") ON DELETE CASCADE,
        "vote" SMALLINT NOT NULL,
        CONSTRAINT "vote_either_1_up_or_minus1_down" CHECK("vote" IN (1, -1)),
        CONSTRAINT "unique_vote_on_post_per_user" UNIQUE("user_id", "post_id")
    );
    CREATE INDEX "fast_search_of_votes_on_a_post" ON "votes" ("post_id");

## Migrate the data

### Migrating to the user table
    INSERT INTO "users" ("username") SELECT DISTINCT username FROM bad_posts
    UNION
    SELECT REGEXP_SPLIT_TO_TABLE("upvotes", ',') username FROM bad_posts
    UNION 
    SELECT REGEXP_SPLIT_TO_TABLE("downvotes", ',') username FROM bad_posts
    UNION 
    SELECT username FROM bad_comments;
    
### Migrating to the topics table
    INSERT INTO "topics" ("name") SELECT DISTINCT "topic" FROM "bad_posts";
    
### Migrating to the posts table
    INSERT INTO "posts" ("id", "user_id", "topic_id", "title", "url", "text_content") SELECT p.id, u.id user_id, t.id topic_id, p.title, p.url, p.text_content 
    FROM bad_posts p
    JOIN users u
    ON u.username = p.username
    JOIN topics t
    ON p.topic = t.name
    WHERE LENGTH(p.title) <= 100;
    
### Migrating to comments table
    INSERT INTO "comments" ("id", "user_id", "post_id", "text_content") SELECT c.id, u.id user_id, c.post_id, c.text_content 
    FROM bad_comments c
    JOIN users u
    ON u.username = c.username
    JOIN posts p
    ON c.post_id = p.id;
    
### Migrating to votes table
    INSERT INTO "votes" ("user_id", "post_id", "vote") SELECT u.id user_id, sub.id post_id, sub.vote
    FROM (
        SELECT REGEXP_SPLIT_TO_TABLE("upvotes", ',') username, id, 1 vote 
        FROM bad_posts
        UNION
        SELECT REGEXP_SPLIT_TO_TABLE("downvotes", ',') username, id, -1 vote 
        FROM bad_posts
    ) sub
    JOIN users u
    ON u.username = sub.username
    JOIN posts p
    ON p.id = sub.id;
